## TRABALHO DE CONCLUSÃO DE CURSO
Agente de IA para vender e falar sobre o Método CIS, que é um produto da empresa

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.0 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.2 MB/s eta 0:00:00


In [ ]:
# === Melhor Extrator de PDF (PyMuPDF + fallback pdfplumber) ===
# - Mantém layout básico e quebra de linha
# - Remove hífens de quebra de linha opcionais
# - Gera objetos Document compatíveis com LangChain (page_content, metadata)

import re
from typing import List
import os

# PyMuPDF (fitz) para extração primária, pdfplumber como fallback
import fitz  # PyMuPDF
import pdfplumber

try:
    from langchain.docstore.document import Document
except Exception:
    # Fallback leve: estrutura similar a Document (mantém compatibilidade básica)
    class Document(dict):
        def __init__(self, page_content: str, metadata: dict = None):
            super().__init__(page_content=page_content, metadata=metadata or {})
        @property
        def page_content(self):
            return self["page_content"]
        @property
        def metadata(self):
            return self["metadata"]

def _clean_text(text: str, dehyphen: bool = True) -> str:
    # Normaliza espaços e quebras
    text = text.replace('\r', '')
    # Junta hífens de quebra de linha: "pal-\navra" -> "palavra"
    if dehyphen:
        text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    # Condensa múltiplas quebras em no máx. 2
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

def extract_pdf_documents(path: str, dehyphen: bool = True) -> List[Document]:
    docs: List[Document] = []
    # Prefer PyMuPDF
    try:
        with fitz.open(path) as pdf:
            for i, page in enumerate(pdf):
                # "text" preserva ordem de blocos melhor que "blocks" na maioria dos casos
                text = page.get_text("text") or ""
                text = _clean_text(text, dehyphen=dehyphen)
                docs.append(Document(page_content=text, metadata={"source": os.path.abspath(path), "page": i + 1}))
        return docs
    except Exception as e:
        # Fallback para pdfplumber
        with pdfplumber.open(path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                text = _clean_text(text, dehyphen=dehyphen)
                docs.append(Document(page_content=text, metadata={"source": os.path.abspath(path), "page": i + 1}))
        return docs


In [ ]:
# Instalando FAISS (biblioteca mais usada)
!pip install openai sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 31.6 MB/s eta 0:00:00


In [ ]:
# importando as bibliotecas que serão usadas
import openai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 🔑 Configuração da API
# A API key agora pode ser definida como uma variável de ambiente OPENAI_API_KEY
# Chave API real
api_key = "#chave API" # <--- SUBSTITUA POR SUA CHAVE API REAL

# 🔤 Simulando uma base de conhecimento para iniciar o modelo
knowledge_base = [
    "O Método CIS, desenvolvido por Paulo Vieira, significa Método do Coaching Integral sistêmico.",
    "O Método CIS é o maior treinamento de inteligência emocional do mundo e já impactou mais de um milhão e meio de pessoas.",
    "O Método CIS já foi transmitido em 83 países e 4 continentes"
   ]



In [ ]:
# 🧠 Modelo de embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
kb_embeddings = embedder.encode(knowledge_base, convert_to_numpy=True)

# 🎯 Indexação FAISS
index = faiss.IndexFlatL2(kb_embeddings.shape[1])
index.add(kb_embeddings)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# TESTANDO O MODELO

# ❓ Pergunta do usuário
question = "Me explique o que é o Método CIS?"

# 🧲 Embedding da pergunta
question_embedding = embedder.encode([question], convert_to_numpy=True)

# 🔍 Recuperando contexto relevante
top_k = 1
_, retrieved_indices = index.search(question_embedding, top_k)
retrieved_context = knowledge_base[retrieved_indices[0][0]]

# Criar uma instância do cliente OpenAI com a nova sintaxe
client = openai.OpenAI(api_key=api_key)


In [ ]:
# 📤 Prompt COM RAG
rag_prompt = f"Contexto: {retrieved_context}\n\nPergunta: {question}"
# Usar o novo método client.chat.completions.create
response_com_rag = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": rag_prompt}]
)



In [ ]:
print("🟢 COM RAG:\n", response_com_rag.choices[0].message.content)

🟢 COM RAG:
 O Método CIS é um programa de treinamento que tem como objetivo ajudar as pessoas a desenvolverem e fortalecerem a inteligência emocional. Por meio de palestras, workshops e atividades práticas, os participantes aprendem a lidar de maneira mais eficaz com suas emoções, aprimorar suas habilidades de comunicação, desenvolver autoconfiança e autocontrole, entre outros aspectos. O programa já impactou mais de um milhão e meio de pessoas em todo o mundo, ajudando-os a alcançarem uma vida mais equilibrada e realizada.


##vamos usar langchain para fazer um "rag" com pdf

In [ ]:
!pip install langchain langchain-community

In [ ]:
!pip install chromadb

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA


# Carregar dois arquivos PDF
docs_from_pdf1 = extract_pdf_documents("/content/Febracis_merged.pdf")
docs_from_pdf2 = extract_pdf_documents("/content/MétodoCIS_merged.pdf")

# Concatenar em uma única lista de documentos
documents = docs_from_pdf1 + docs_from_pdf2

# 2. Dividindo os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs_chunked = text_splitter.split_documents(documents)

# 3. Criando o indexador
embedding = OpenAIEmbeddings(openai_api_key=api_key)
db = Chroma.from_documents(docs_chunked, embedding)

# 4. Criando o sistema RAG com retriever + LLM
retriever = db.as_retriever(search_kwargs={"k": 3})
llm = OpenAI(temperature=0.5, openai_api_key=api_key)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 5. Testando
pergunta = "O que é o Método CIS"
resposta = rag_chain.run(pergunta)
print(resposta)

/tmp/ipython-input-4001841048.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=api_key)
/tmp/ipython-input-4001841048.py:25: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.5, openai_api_key=api_key)
/tmp/ipython-input-4001841048.py:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0

 O Método CIS é o maior treinamento de Inteligência Emocional do mundo, que trabalha os principais pilares da vida de um ser humano e já transformou a vida de 1,5 milhões de pessoas em todo o mundo ao longo de mais de 20 anos.


In [ ]:
# prompt: aumento de temperatura da célula anterior

# Aumentar a temperatura geralmente torna as respostas mais criativas/aleatórias.
llm = OpenAI(temperature=1, openai_api_key=api_key)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 4. Testando (com a nova temperatura)
pergunta = "O que é Método CIS? Explique com detalhes"
resposta = rag_chain.run(pergunta)
resposta

' Método CIS é o maior treinamento de Inteligência Emocional do mundo que trabalha os principais pilares da vida de um ser humano. O treinamento utiliza ferramentas e conceitos, comprovados por pesquisadores da neurociência e do comportamento, para promover mudanças e eliminar barreiras que impedem as pessoas de conquistar tudo aquilo que merecem. Ao longo de mais de 20 anos, o Método CIS transformou a vida de 1,5 milhões de pessoas em todo o mundo, oferecendo a elas uma compreensão melhor das emoções e ajudando-as a lidar com elas de forma mais eficaz e positiva.'